# Cryptography (CC4017) -- Week 10 - Extra 

## Q1: Man-in-the-middle attack

Implement a prototype that demonstrates how a Man-in-the-Middle attack can occur in a standardunauthenticated Diffie-Hellman key exchange. As usual, this happens between usual suspects Alice and Bob. In appendix, you can find four files:

- config_alice and config_bob are configuration files that are used by Alice and Bob to know who to talk to.

- alice.py establishes a connection with Bob (hopefully), and performs the Diffie-Hellman key exchange.
    
- bob.py mirrors the behavior of Alice.
    
The goal of the work is to design the man-in-the-middle adversary code:mitm.py. Your code must convince Alice and Bob to instead talk to him, and perform a key exchange with him. Your attack must not change the source code of Alice or Bob. Your attack is successful if Alice and Bob are not agreeing on the same secret, and the secrets they have agreed to are both known to the adversary.
    
**Suggestion:** Start by analysing the code for Alice and Bob, what are they using to communicate? How can we subvert this mechanism to be more. . . convenient?
    
To facilitate communication, this code usespwntools(reference). It is not mandatory to use this, butthe library considerably facilitates communication

In [ ]:
from pwn import * # type: ignore

# Load configurations for Alice and Bob
with open("config_alice", "r") as f_alice:
    hostBob = f_alice.readline().strip()  # Bob's host from Alice's config
    portBobOut = int(f_alice.readline().strip())  # Bob's port from Alice's config

with open("config_bob", "r") as f_bob:
    hostAlice = f_bob.readline().strip()  # Alice's host from Bob's config
    portAliceOut = int(f_bob.readline().strip())  # Alice's port from Bob's config

# MITM interception ports
hostMITM = "localhost"
portAliceIn = 5075  # Port MITM listens to intercept Alice
portBobIn = 5076    # Port MITM listens to intercept Bob

# Diffie-Hellman parameters
g = 2
p = 7853799659

# MITM secrets
malicious_y = random.randint(1, p)  # type: ignore # For Alice
malicious_x = random.randint(1, p)  # type: ignore # For Bob

# Connections for interception
lAlice = listen(portAliceIn) # type: ignore
lAlice.wait_for_connection()
rBob = remote(hostBob, portBobOut) # type: ignore

lBob = listen(portBobIn) # type: ignore
lBob.wait_for_connection()
rAlice = remote(hostAlice, portAliceOut) # type: ignore

# Intercept Alice's GX
gx_alice = int.from_bytes(lAlice.recvline()[:-1], "little")
print("[MITM] Received GX from Alice:", gx_alice)

gy_mitm_for_bob = pow(g, malicious_y, p)
print("[MITM] Sending GY to Bob:", gy_mitm_for_bob)
rBob.sendline(gy_mitm_for_bob.to_bytes(8, "little"))

# Intercept Bob's GY
gy_bob = int.from_bytes(lBob.recvline()[:-1], "little")
print("[MITM] Received GY from Bob:", gy_bob)

gx_mitm_for_alice = pow(g, malicious_x, p)
print("[MITM] Sending GX to Alice:", gx_mitm_for_alice)
rAlice.sendline(gx_mitm_for_alice.to_bytes(8, "little"))

# Compute shared secrets
shared_secret_alice = pow(gx_alice, malicious_y, p)
shared_secret_bob = pow(gy_bob, malicious_x, p)

print("[MITM] Shared secret with Alice:", shared_secret_alice)
print("[MITM] Shared secret with Bob:", shared_secret_bob)

# Close connections
lAlice.close()
rBob.close()
lBob.close()
rAlice.close()


## Q2: ECC

The following is a naive attempt at an elliptic curve signature scheme. Consider a global elliptic curve, prime $p$ and generator $G$. The scheme works as follows.

•Alice picks a private signing key $sk_A$ and forms the public verifying key by computing $pk_A ← sk_A·G$

•To sign messagem, Alice picks a random valuek, and computes the signature $σ←m−k·sk_A·G$.It then sends to Bob the tuple(m,k,σ)

• To verify the signature, Bob checks that $m=σ+k·pk_A$. 

If this is true, the signature is validated.

## Question - P1:
Show that the scheme works, i.e. show that, for correctly signed messages, theverification algorithm works accordingly.

### Answer to P1:
>**Given:**
>- Alice's private signing key: $ sk_A $
>- Alice's public verifying key: $ pk_A = sk_A \cdot G $
>- Message: $ m $
>- Random value: $ k $
>- Signature: $ \sigma = m - k \cdot sk_A \cdot G $
>
>Alice sends the tuple $ (m, k, \sigma) $ to Bob.
>
>**Verification:**
>
>Bob checks if $ m = \sigma + k \cdot pk_A $.
>
>**Let's substitute the values:**
>
>$ \sigma = m - k \cdot sk_A \cdot G $
>
>$ k \cdot pk_A = k \cdot (sk_A \cdot G) $
>
>**So,**
>
>$ \sigma + k \cdot pk_A = (m - k \cdot sk_A \cdot G) + k \cdot (sk_A \cdot G) $
>
>$ \sigma + k \cdot pk_A = m - k \cdot sk_A \cdot G + k \cdot sk_A \cdot G $
>
>$ \sigma + k \cdot pk_A = m $
>
>Thus, the verification algorithm holds true for correctly signed messages.

## Question - P2:
Show that this scheme is vulnerable, by describing a simple technique for forginga signature on an arbitrary message, without knowledge of the secret key $sk_A$.
Hint: Consider what computations can one do using simply $pk_A$

### Answer to P2:

>Given:
>- Alice's public verifying key: $ pk_A = sk_A \cdot G $
>
>Let's consider an arbitrary message $ m' $. We want to forge a signature $ >\sigma' $ and a value $ k' $ such that the verification equation holds:
>$ m' = \sigma' + k' \cdot pk_A $
>
>We can choose $ k' $ arbitrarily, for example, $ k' = 1 $.
>
>Then, we can compute $ \sigma' $ as:
>$ \sigma' = m' - k' \cdot pk_A $
>
>$ \sigma' = m' - 1 \cdot pk_A $
>
>$ \sigma' = m' - pk_A $
>
>Now, we have:
>$ m' = \sigma' + k' \cdot pk_A $
>
>$ m' = (m' - pk_A) + 1 \cdot pk_A $
>
>$ m' = m' - pk_A + pk_A $
>
>$ m' = m' $
>
>Thus, we have successfully forged a signature $ \sigma' $ for the arbitrary message $ m' $ without knowing the secret key $ sk_A $.
>
>This demonstrates that the scheme is vulnerable to forgery.

## Q3: ELGamal

ElGamal is a public-key encryption scheme. Its reasoning is similar to that of classical Diffie-Hellman, using $g^x$ as the public key, and having the encryption encapsulate the message with $g^y$. The algorithms are presented below, assuming operations in the group $Z_q$, with generatorg. Encryption assumes thatmis an element of $Z_q$, whcih can be achieved by having a reversible mapping function from the message domain to the group domain. $s^{−1}$ means the inverse of s.

**Algorithm Gen():**

$x ←\$ \{1, ..., (q - 1)\}$

$X ← g^x$

$Return \ (X, x)$

**Algorithm Enc(X, m):**

$y ←\$ \{1, ..., (q - 1)\}$

$s ← y^X$

$c ← g^y, c ← m * s$

$Return \ (Y, c)$

**Algorithm Dec(x, Y, c):**
$s ← Y^x$

$m ← c * s^-1$

$Return \ m$

## Question - P1:

Describe why decryption works – show how $m$ is recovered

>1. **Encryption:**
>
>   - Choose a random $y $ from $\{1, ..., (q - 1)\} $.
>
>   - Compute $ s = X^y $
>
>   - Compute $ c_1 = g^y $
>
>   - Compute $ c_2 = m \cdot s $
>
>   - The ciphertext is $ (c_1, c_2) $.
>
>2. **Decryption:**
>
>  - Compute $ s = c_1^x $.
>   
>  - Compute $s^{-1} $, the inverse of $s $
>
>   - Recover the message $ m = c_2 \cdot s^{-1} $.
>
>Steps in detail:
>
>- During encryption, $ s = X^y = (g^x)^y = g^{xy} $.
>
>- The ciphertext is $ (c_1, c_2) = (g^y, m \cdot g^{xy}) $.
>
>During decryption:
>- Compute $ s = c_1^x = (g^y)^x = g^{xy} $.
>
>- Compute $ s^{-1} $, the inverse of $ g^{xy} $.
>
>- Recover the message $ m = c_2 \cdot s^{-1} = (m \cdot g^{xy}) \cdot (g^{xy})^{-1} = m $.
>
>Thus, the original message $ m $ is recovered.

## Question - P2:

Explain how the hardness of the discrete logarithm ensures confidentiality

> The hardness of the discrete logarithm problem ensures confidentiality in the ElGamal encryption scheme. 
>
> Specifically, the security relies on the difficulty of solving the following problems:
>
>1. **Discrete Logarithm Problem (DLP):** Given $g $ and $ g^x $ it is computationally infeasible to determine $x $
>
>2. **Computational Diffie-Hellman Problem (CDHP):** Given $ g^x $ and $ g^y $ it is computationally infeasible to determine $ g^{xy} $.
>
>
> In ElGamal encryption:
>
> - The public key $ X = g^x $ is derived from the private key $ x $
> 
> - The ciphertext includes $c_1 = g^y $ and $ c_2 = m \cdot g^{xy} $.
>
> An adversary who intercepts the ciphertext $ (c_1, c_2) $ would need to solve the DLP to find $ y $ from $ g^y $ or solve the CDHP to find $ g^{xy} $ from $ g^x $ and $ g^y $. Both problems are considered hard, ensuring that the adversary cannot feasibly recover the message $ m $ without the private key $ x $.

## Question - P3:

ElGamal is malleable – show how it can be done. Consider an adversary that can request the encryption of message $m$ , receiving ciphertext $c$, and show how can he present another ciphertext $c'$ that will decrypt in a related way.

>ElGamal encryption is malleable, meaning that an adversary can modify a ciphertext to produce a new ciphertext >that decrypts to a related message. Here's how it can be done:
>
>1. **Original Ciphertext:**
>   - Suppose the adversary intercepts the ciphertext $(c_1, c_2) = (g^y, m \cdot g^{xy}) $
>
>2. **Modification:**
>
>     - The adversary can choose a value $k $ and compute a new ciphertext $(c_1', c_2') $ as follows:
>
>     - $ c_1' = c_1 \cdot g^k = g^y \cdot g^k = g^{y+k} $.
>
>     - $c_2' = c_2 \cdot g^{kx} = (m \cdot g^{xy}) \cdot g^{kx} = m \cdot g^{xy} \cdot g^{kx} = m \cdot g^{x(y+k)} $
>
>3. **Decryption of Modified Ciphertext:**
>   - When the recipient decrypts $ (c_1', c_2') $
>
>     - Compute $s' = (c_1')^x = (g^{y+k})^x = g^{x(y+k)} $.
>
>     - Compute $ s'^{-1} $, the inverse of $g^{x(y+k)} $.
>
>     - Recover the message $ m' = c_2' \cdot s'^{-1} = (m \cdot g^{x(y+k)}) \cdot (g^{x(y+k)})^{-1} = m $.
>
> Thus, the adversary has created a new ciphertext $ (c_1', c_2') $ that decrypts to the same message $ m $, demonstrating the malleability of ElGamal encryption.